# Send request to filter example

To use this example, start a Diffusion server and update the connection details below as needed.

The example creates two sessions, sets a request handler on one of them (the receiver) and the filter response handler on the other (the sender). The sender then sends a message to the filter, and waits to receive a response from the receiver.

In [ ]:
import asyncio
import diffusion

In [ ]:
# Diffusion server connection information; same for both sessions
# adjust as needed for the server used in practice
server_url = "ws://localhost:8080"
principal = "admin"
credentials = diffusion.Credentials("password")


# handler callback function
def request_handler(request: str, context=None) -> str:
    return f"Hello there, {request}!"


# filter response handler function
def on_filter_response(response, context=None):
    print(f"Received response from session '{context['session_id']}':")
    print(f"    {response}")
    print(" - Request was sent to {filter} on path {path}".format(**context))
    print(" - Received {received} of {expected} response(s).".format(**context))


# request properties
request = "Pushme Pullyou"
path = "path"
request_type = diffusion.datatypes.STRING  # datatype of the request

In [ ]:
# creating the two sessions
sender_session = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)
receiver_session = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)

async with sender_session, receiver_session:
    # instantiating the messaging components for both sessions
    receiver = diffusion.Messaging(receiver_session)
    sender = diffusion.Messaging(sender_session)

    # adding the handler for addressed requests
    receiver.add_request_handler(
        path,
        callback=request_handler,
        request_type=request_type,
        response_type=request_type,
        addressed=True,
    )

    # specifying the session filter
    # this is a very simple filter, addressing the receiver session directly
    session_filter = f"$SessionId is '{receiver_session.session_id}'"

    # adding filter response handler
    sender.add_filter_response_handler(
        session_filter=session_filter, callback=on_filter_response
    )

    # sending the request and receiving the number of expected responses
    print(f"Sending request: '{request}' to session filter `{session_filter}`...")
    try:
        response = await sender.send_request_to_filter(
            session_filter=session_filter, path=path, request=request_type(request),
        )
    except diffusion.DiffusionError as ex:
        print(f"ERROR: {ex}")
    else:
        print(f"... expecting {response} response(s) ...")

    # waiting a bit to receive all responses
    await asyncio.sleep(1)


In [ ]:
request = "Pushme Pullyou"
path = "path"
request_type = diffusion.datatypes.STRING  # datatype of the request

In [ ]:
# creating the two sessions
sender_session = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)
receiver_session = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)

async with sender_session, receiver_session:
    # instantiating the messaging components for both sessions
    receiver = diffusion.Messaging(receiver_session)
    sender = diffusion.Messaging(sender_session)

    # adding the handler for addressed requests
    receiver.add_request_handler(
        path,
        callback=request_handler,
        request_type=request_type,
        response_type=request_type,
        addressed=True,
    )

    # specifying the session filter
    # this is a very simple filter, addressing the receiver session directly
    session_filter = f"$SessionId is '{receiver_session.session_id}'"

    # adding filter response handler
    sender.add_filter_response_handler(
        session_filter=session_filter, callback=on_filter_response
    )

    # sending the request and receiving the number of expected responses
    print(f"Sending request: '{request}' to session filter `{session_filter}`...")
    try:
        response = await sender.send_request_to_filter(
            session_filter=session_filter, path=path, request=request_type(request),
        )
    except diffusion.DiffusionError as ex:
        print(f"ERROR: {ex}")
    else:
        print(f"... expecting {response} response(s) ...")

    # waiting a bit to receive all responses
    await asyncio.sleep(1)
